In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense
import numpy as np

# Define your inputs X and outputs y
X = np.array([
    {'play': 45, 'action': 1, 'offense_power': 25, 'defense_power': 30},
    {'play': 46, 'action': 3, 'offense_power': 25, 'defense_power': 30}
])

y = np.array([
    [{'play': 1, 'yards_gained': 25}, {'play': 2, 'yards_gained': 0}],
    [{'play': 1, 'yards_gained': 1}, {'play': 2, 'yards_gained': 10}]
])

# Define the input layers
play_input = Input(shape=(1,))
action_input = Input(shape=(1,))
offense_power_input = Input(shape=(1,))
defense_power_input = Input(shape=(1,))

# Define the embedding layer for action
embedding_dim = 8
action_embed = Embedding(input_dim=4, output_dim=embedding_dim)(action_input)

# Combine the input layers and action embedding
combined_input = Dense(32, activation='relu')([play_input, action_embed, offense_power_input, defense_power_input])
# Add more layers as needed

# Define the output layers
play_output = Dense(1, activation='linear')(combined_input)
yards_gained_output = Dense(1, activation='linear')(combined_input)

# Create the model
model = Model(inputs=[play_input, action_input, offense_power_input, defense_power_input],
              outputs=[play_output, yards_gained_output])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, [y[:,0], y[:,1]], epochs=10, batch_size=1)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
import tensorflow.keras.backend as K

# Define input layer
inputs = Input(shape=(num_features,))

# Shared layers for feature extraction
shared_layer_1 = Dense(64, activation='relu')(inputs)
shared_layer_2 = Dense(32, activation='relu')(shared_layer_1)

# Task-specific layers for YARDS prediction
yards_layer = Dense(16, activation='relu')(shared_layer_2)
yards_output = Dense(1, name='yards_output')(yards_layer)

# Task-specific layers for POINTS prediction
points_layer = Dense(16, activation='relu')(shared_layer_2)
points_output = Dense(1, name='points_output')(points_layer)

# Define the model with multiple outputs
model = Model(inputs=inputs, outputs=[yards_output, points_output])

# Define loss functions and metrics for each output
model.compile(optimizer='adam',
              loss={'yards_output': 'mean_squared_error',
                    'points_output': 'binary_crossentropy'},
              metrics={'yards_output': 'mae',
                       'points_output': 'accuracy'})

# Train the model with multiple targets
model.fit(X_train, {'yards_output': y_train_yards, 'points_output': y_train_points}, epochs=10, batch_size=32)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model architecture
class MultiTaskModel(nn.Module):
    def __init__(self, num_features):
        super(MultiTaskModel, self).__init__()
        self.shared_layer_1 = nn.Linear(num_features, 64)
        self.shared_layer_2 = nn.Linear(64, 32)
        self.yards_layer = nn.Linear(32, 16)
        self.yards_output = nn.Linear(16, 1)
        self.points_layer = nn.Linear(32, 16)
        self.points_output = nn.Linear(16, 1)

    def forward(self, x):
        shared = torch.relu(self.shared_layer_1(x))
        shared = torch.relu(self.shared_layer_2(shared))
        yards = torch.relu(self.yards_layer(shared))
        yards = self.yards_output(yards)
        points = torch.relu(self.points_layer(shared))
        points = self.points_output(points)
        return yards, points

# Create an instance of the model
model = MultiTaskModel(num_features)

# Define loss functions
yards_criterion = nn.MSELoss()
points_criterion = nn.BCELoss()

# Define optimizer
optimizer = optim.Adam(model.parameters())

# Train the model with multiple targets
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    yards_pred, points_pred = model(X_train)
    yards_loss = yards_criterion(yards_pred, y_train_yards)
    points_loss = points_criterion(points_pred, y_train_points)
    loss = yards_loss + points_loss
    loss.backward()
    optimizer.step()
